In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
from sklearn.neural_network import MLPClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [ ]:
dog_bark = []
say_go = []
baby_cry = []

In [ ]:
def draw_hist(folder):
    d = []
    f = os.path.join('/kaggle/input/freesound-audio-ds-project/ProcessedAudio/' ,folder)
    reprocess = []
    for filename in os.listdir(f):
        try:
            y, sr = librosa.load(os.path.join(f, filename))
            du = librosa.get_duration(y, sr)
            d.append(du)
        except:
            continue
    plt.hist(d, bins = 20)
    plt.show()

In [8]:
labels = ['dog bark', 'say go', 'baby cry']

In [11]:
def get_dataset(folder):
    X = []
    ys = []
    f = os.path.join('/kaggle/input/preprocess-audio/' ,folder)
    for filename in os.listdir(f):
        try:
            y, sr = librosa.load(os.path.join(f, filename))
        except:
            print(folder + "/" + filename)
        X.append((y, sr))
        ys.append(int(labels.index(folder)))
    return X, ys

In [12]:
# Get all data
X = []
y = []
for folder in labels:
    X_folder, y_folder = get_dataset(folder)
    X += X_folder
    y += y_folder

baby cry/252.wav
baby cry/253.wav


In [13]:
class FixLength(BaseEstimator, TransformerMixin):
    def __init__(self, length):
        self.length = length
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # X contains tupes of y and sr
        res = []
        for y, sr in X:
            y_fixed = librosa.util.fix_length(y, self.length)
            res.append((y_fixed, sr))
        return res

NameError: name 'BaseEstimator' is not defined